# Hippocratech Healthcare Data

_April 21st, 2019_

Datasets are too large to be uploaded to GitHub. So only a final version of cleaned and combined dataset is uploaded here, which contains datasets for quarter 4 from 2012 to 2017.

## Table of Contents

<div class='alert alert-block alert-info' style='margin-top: 20px'>
    <li><a href='#ref1'>1. Data Preprocessing</a>
    <li><a href='#ref2'>2. Map Zip to Sub-Borough</a>
        <li><a href='#ref21'>---- 2.1 Initial mapping</a>
        <li><a href='#ref22'>---- 2.2 Check missing zip codes</a>
        <li><a href='#ref23'>---- 2.3 Map missing zip codes to PUMA</a>
        <li><a href='#ref24'>---- 2.4 Impute data based on maps generated in 2.3</a>
        <li><a href='#ref25'>---- 2.5 Check missing zip codes after imputation</a>
        <li><a href='#ref26'>---- 2.6 Map Zip to Sub-Borough</a>
</div>

<a id='ref1'></a>
## 1. Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import zipfile

In [2]:
def read_data(year, zipname, csvname):
    zf = zipfile.ZipFile('./data/%s' % zipname)
    
    if year in [2017, 2016]:
        data = pd.read_csv(zf.open('%s' % csvname),
                           usecols = ['SiteName_std', 'City_std', 'State_std', 'Zip_std', 
                                      'TrueCountyName', 'desser'], # desser - Designated Service
                           dtype = {'Zip_std': object})
        data.rename(columns={'SiteName_std':'sitename', 'City_std':'city', 'State_std':'state', 
                             'Zip_std':'zip', 'TrueCountyName':'county'}, inplace=True)
        # inplace - Whether to return a new DataFrame. If True then value of copy is ignored.
        data.rename(str.lower, axis='columns', inplace=True)
    
    elif year in [2015, 2014]:
        data = pd.read_csv(zf.open('%s' % csvname),
                           usecols = ['sitename', 'city', 'state', 'zip', 'county', 'desser'],
                           dtype = {'zip': object})
        
    elif year==2013:
        data = pd.read_csv(zf.open('NYSDOH_PNDS_InstitutionalProviderData_2013Q4.csv'),
                           usecols = ['SITE', 'city', 'state', 'zip', 'county', 'desser'],
                           dtype = {'zip': object}, encoding='cp1252')
        data.rename(columns={'SITE':'sitename'}, inplace=True)
        
    elif year==2012:
        data = pd.read_csv(zf.open('NYSDOH_PNDS_InstitutionalProviderData_2012Q4.csv'),
                           usecols = ['sitename', 'City', 'State', 'Zip', 'county', 'Desser'],
                           dtype = {'Zip': object}, encoding='cp1252')
        data.rename(str.lower, axis='columns', inplace=True)
    
    data['year'] = year
    
    return data

The counties New York City are:

005 - Bronx

047 - Kings (Brooklyn)

061 - New York (Manhattan)

081 - Queens

085 - Richmond (Staten Island)

In [3]:
def clean_data(year, data):    
    # 1. New York State
    data_ny = data[data.state == 'NY']
    
    # 2. New York City
    data_ny2 = data_ny.copy()
    if year in [2015, 2014, 2013, 2012]:
        data_ny2.loc[:,['county']] = data_ny.county.replace({5: 'Bronx', 47: 'Kings', 61: 'New York', 
                                                 81: 'Queens', 85: 'Richmond'})
    
    nyc_counties = ['Bronx', 'Kings', 'New York', 'Queens', 'Richmond']
    data_nyc = data_ny2[data_ny2.county.isin(nyc_counties)]
    
    # 3. Check services and remove pharmacy
    data_noPharma = data_nyc[~(data_nyc.desser == 760)]
    
    return data_noPharma

In [6]:
zip_and_csv = [[2017, 
                'NYSDOH_PNDS_InstitutionalProviderData_2017Q4.zip', 
                'PNDS_Institutional_Q417.csv'],
               [2016, 
                'NYSDOH_PNDS_InstitutionalProviderData_2016_Q04.zip', 
                'PNDS_Institutional_Q416_v2.csv'],
               [2015, 
                'NYSDOH_PNDS_InstitutionalProviderData_2015_Q04.zip', 
                'NYSDOH_PNDS_InstitutionalProviderData_2015_Q04.csv'],
               [2014, 
                'NYSDOH_PNDS_InstitutionalProviderData_2014Q4.zip',
                'NYSDOH_PNDS_InstitutionalProviderData_2014Q4.csv'],
               [2013,
                'NYSDOH_PNDS_InstitutionalProviderData_2013Q4.zip',
                'NYSDOH_PNDS_InstitutionalProviderData_2013Q4.csv'],
               [2012,
                'NYSDOH_PNDS_InstitutionalProviderData_2012Q4.zip',
                'NYSDOH_PNDS_InstitutionalProviderData_2012Q4.csv']]

In [75]:
%%time
# Combine datasets from 2012 to 2017
appended_data = []
for i in zip_and_csv:
    readdata = read_data(year=i[0], zipname=i[1], csvname=i[2])
    cleandata = clean_data(year=i[0], data=readdata)
    values = {"foo1": i[0], "foo2": len(cleandata)}
    print('Year {foo1} has a total number of {foo2} of institutional providers'.format(**values))
    appended_data.append(cleandata)
    
data12to17 = pd.concat(appended_data, sort=True)

# Remove duplicates
data12to17_noDup = data12to17.drop_duplicates()

Year 2017 has a total number of 157902 of institutional providers
Year 2016 has a total number of 74520 of institutional providers
Year 2015 has a total number of 111796 of institutional providers
Year 2014 has a total number of 68928 of institutional providers
Year 2013 has a total number of 53163 of institutional providers
Year 2012 has a total number of 44811 of institutional providers
CPU times: user 16.4 s, sys: 1.52 s, total: 17.9 s
Wall time: 16.4 s


In [76]:
# Add a column borough
def countyTOborough(row):
    if row['county'] == 'Kings':
        return 'Brooklyn'
    if row['county'] == 'New York':
        return 'Manhattan'
    if row['county'] == 'Richmond':
        return 'Staten Island'
    if row['county'] == 'Bronx':
        return 'Bronx'
    if row['county']  == 'Queens':
        return 'Queens'

In [77]:
data12to17_noDup = data12to17_noDup.assign(borough = data12to17_noDup.apply(lambda row: countyTOborough(row), axis=1))

In [79]:
# Export to csv file
data12to17_noDup.to_csv('./data/institutional_provider_2012to2017_q4.csv')

<a id='ref2'></a>
## 2. Map Zip to Sub-Borough
One sub-borough can have several zip codes, and a zip code can belong to multiple sub-boroughs as well, e.g. 11238.
Here we'll assign a zip code to only one sub-boroughs when it first appears in the `Zip_Code_Data.xlsx` dataset.

<a id='ref21'></a>
### 2.1 Initial mapping

In [214]:
zipdata = pd.read_excel('./data/Zip_Code_Data.xlsx', dtype = {'Zip codes': str})

In [217]:
zipdata = zipdata.drop([46, 47, 48])

In [229]:
def zipTOsubborough(row):
    for i in range(len(zipdata)):
        if row['zip'] in zipdata.loc[i, :]['Zip codes']:
            return zipdata.loc[i, :]['Sub-Borough']

In [235]:
data1217 = data12to17_noDup.assign(sub_borough = data12to17_noDup.apply(lambda row: zipTOsubborough(row), axis=1))

In [236]:
data1217.head()

,city,county,desser,sitename,state,year,zip,borough,sub_borough
1,BRONX,Bronx,663.0,LACONIA NURSING HOME,NY,2017,10466,Bronx,Williamsbridge/Baychester
8,BROOKLYN,Kings,668.0,CITY CHOICE HOME CARE SERVICES,NY,2017,11235,Brooklyn,None
9,BROOKLYN,Kings,914.0,NYU LANGONE MED CENTER LEVIT MED ARTS PAVILLION,NY,2017,11236,Brooklyn,East New York/Starrett City
10,BROOKLYN,Kings,680.0,"SUMMIT HOME HEALTH CARE, INC.",NY,2017,11230,Brooklyn,Borough Park
19,FAR ROCKAWAY,Queens,375.0,TIKVAH AT OHEL,NY,2017,11691,Queens,None


In [366]:
data1217.sub_borough.value_counts(dropna=False).head()

NaN                             42824
Bensonhurst                      7609
Chelsea/Clinton/Midtown          6869
Brooklyn Heights/Fort Greene     5993
Flushing/Whitestone              5945
Name: sub_borough, dtype: int64

A lot of zip codes don't belong to any sub-borough. The `Zip_Code_Data.xlsx` might be incomplete.

In [240]:
# Export to csv file
data1217.to_csv('./data/institutional_provider_2012to2017_q4_subborough.csv')

<a id='ref22'></a>
### 2.2 Check missing zip codes

In [301]:
mapzip = []
for i in zipdata['Zip codes']:
    j = i.split()
    mapzip += i.split()

sizes = pd.DataFrame(data={'zip': mapzip, 'size': [1]*len(mapzip)})

import folium
#Center the map at Times Square
m = folium.Map(location=[40.7589, -73.9851], zoom_start=11)
m.choropleth(geo_data='./data/zipcode.geojson', data=sizes,
             columns=['zip','size'],
             key_on='feature.properties.postalCode',
             fill_color='RdYlGn', fill_opacity=0.7, line_opacity=0.8,
             legend_name='Check missing zip codes')
folium.LayerControl().add_to(m)

/Users/Ivy_Zhang/anaconda3/lib/python3.6/site-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [302]:
m

<a id='ref23'></a>
### 2.3 Map missing zip codes to PUMA

[Neighborhood Definition](http://a816-dohbesp.nyc.gov/IndicatorPublic/EPHTPDF/subboro.pdf):
The boundaries of sub-borough areas often approxiamate those of Community Districts.

Public Use Microdata Areas (PUMAs) approximate NYC Community Districts (CDs). ([Link](https://www1.nyc.gov/assets/planning/download/pdf/data-maps/nyc-population/census2010/puma_cd_map.pdf))

So I use PUMA to approximate sub-borough.

In [422]:
m = folium.Map(location=[40.7589, -73.9851], zoom_start=11, tiles='Mapbox Bright')

folium.GeoJson(
    './data/zipcode.geojson',
    name='geojson',
    tooltip=folium.features.GeoJsonTooltip(fields=['postalCode'],
                                    aliases=['Zip Code'],
                                    labels=True, 
                                    sticky=True),
    style_function=lambda x: {'weight':1,'color':'black','dashArray':'1 3','opacity':0.6},
    highlight_function=lambda x: {'weight':1, 'color':'blue'}
).add_to(m)

folium.LayerControl().add_to(m)

In [424]:
m.save('./map/map_zip.html')

Public Use Microdata Areas (PUMA).geojson: https://data.cityofnewyork.us/Housing-Development/Public-Use-Microdata-Areas-PUMA-/cwiz-gcty

Reference:

[show folium outside of ipython notebooks](https://github.com/python-visualization/folium/issues/946#issuecomment-417313698)

[Folium 0.8.3 documentation](https://python-visualization.github.io/folium/modules.html#folium.map.Tooltip)

[adding tooltip to geojson](https://github.com/python-visualization/folium/issues/951#issuecomment-418450802)

[example of tooltip](https://nbviewer.jupyter.org/gist/jtbaker/57a37a14b90feeab7c67a687c398142c?flush_cache=true)

In [419]:
m = folium.Map(location=[40.7589, -73.9851], zoom_start=11)

m.choropleth(geo_data='./data/zipcode.geojson', data=sizes,
             columns=['zip','size'],
             key_on='feature.properties.postalCode',
             fill_color='RdYlGn', fill_opacity=0.5, line_opacity=0.6,
             legend_name='Map Zip Code to PUMA',
)

folium.GeoJson(
    './data/Public Use Microdata Areas (PUMA).geojson',
    name='geojson',
    tooltip=folium.features.GeoJsonTooltip(fields=['puma'],
                                    aliases=['PUMA'],
                                    labels=True, 
                                    sticky=True),
    style_function=lambda x: {'weight':3,'color':'#5BDAE9','dashArray':'1 3','opacity':0.6},
    highlight_function=lambda x: {'weight':3, 'color':'blue'}
).add_to(m)

folium.LayerControl().add_to(m)

/Users/Ivy_Zhang/anaconda3/lib/python3.6/site-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [420]:
m.save('./map/map_subborough_zip.html')

<a id='ref24'></a>
### 2.4 Impute data based on maps generated in 2.3

In [445]:
SubBorough_puma = pd.read_csv('./data/sub_borough_puma.csv', dtype = {'puma': object})
SubBorough_puma = SubBorough_puma.set_index('Sub-Borough')

zipdata = zipdata.set_index('Sub-Borough')

In [673]:
puma_zip = SubBorough_puma.join(zipdata, how='outer')

In [645]:
def impute_data(puma, addzip):
    idx = puma_zip[puma_zip.puma==puma].index
    if puma_zip.loc[idx, 'Zip codes'].isnull().values:
        puma_zip.loc[idx, 'Zip codes'] = [addzip]
    else:
        puma_zip.loc[idx, 'Zip codes'] = puma_zip.loc[idx, 'Zip codes'].values + addzip

In [675]:
allimpute = [['3901', ' 10306 10308 10312 10309 10307'],
             ['3903', ' 10301 10310 10302 10303'],
             ['3902', ' 10314 10304 10305'],
             ['3703', ' 10464 10465 10475'],
             ['3702', ' 10470'],
             ['3705', ' 10460'],
             ['3710', ' 11370'],
             ['4102', ' 11371 11369'],
             ['3806', ' 00083 10069'],
             ['3808', ' 10022 10016 10010'],
             ['3810', ' 10282 10281 10280 10004 10006 10048 10007 10038 10005 10004'],
             ['4004', ' 11251'],
             ['4011', ' 11225'],
             ['4015', ' 11210'],
             ['4016', ' 11229 11235 11223'],
             ['4018', ' 11224'],
             ['4110', ' 11378'],
             ['4103', ' 11359 11360 11358'],
             ['4104', ' 11361 11364 11363 11362'],
             ['4106', ' 11367'],
             ['4111', ' 11415 11418 11419'],
             ['4113', ' 11420 11414'],
             ['4105', ' 11430 11413 11422 11411 11429 11428 11427 11426 11404 11040 11001 11004'],
             ['4114', ' 11693 11697 11694 11692 11691 11096']]

In [676]:
for i in allimpute:
    impute_data(i[0], i[1])

In [677]:
puma_zip = puma_zip.reset_index()

In [684]:
# Export to csv file
puma_zip.to_csv('./data/SubBorough_puma_zip.csv')

<a id='ref25'></a>
### 2.5 Check missing zip codes after imputation

In [679]:
puma_zip['Zip codes'] = puma_zip['Zip codes'].astype('str')

In [680]:
mapzip = []
for i in puma_zip['Zip codes']:
    j = i.split()
    mapzip += i.split()

sizes = pd.DataFrame(data={'zip': mapzip, 'size': [1]*len(mapzip)})

import folium
#Center the map at Times Square
m = folium.Map(location=[40.7589, -73.9851], zoom_start=11)
m.choropleth(geo_data='./data/zipcode.geojson', data=sizes,
             columns=['zip','size'],
             key_on='feature.properties.postalCode',
             fill_color='RdYlGn', fill_opacity=0.7, line_opacity=0.8,
             legend_name='Check missing zip codes')
folium.LayerControl().add_to(m)

/Users/Ivy_Zhang/anaconda3/lib/python3.6/site-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [681]:
m

<a id='ref26'></a>
### 2.6 Map Zip to Sub-Borough

In [685]:
puma_zip = pd.read_csv('./data/SubBorough_puma_zip.csv', dtype = {'Zip codes': str})

In [686]:
def zipTOsubborough_new(row):
    for i in range(len(puma_zip)):
        if row['zip'] in puma_zip.loc[i, :]['Zip codes']:
            return puma_zip.loc[i, :]['Sub-Borough']

In [687]:
data1217_new = data12to17_noDup.assign(sub_borough = data12to17_noDup.apply(lambda row: zipTOsubborough_new(row), 
                                                                            axis=1))

In [690]:
data1217_new.sub_borough.value_counts(dropna=False).tail()

Riverdale/Kingsbridge            756
Soundview/Parkchester            733
South Ozone Park/Howard Beach    714
Sunset Park                      555
NaN                              440
Name: sub_borough, dtype: int64

There're still a small number of zip codes that don't belong to any sub-boroughs.

In [691]:
# Export to csv file
data1217_new.to_csv('./data/institutional_provider_2012to2017_q4_subborough_impute.csv')